In [27]:
OPENSLIDE_PATH = r"C:\Users\MS\openslide-bin-4.0.0.8-windows-x64\bin"

import os

if hasattr(os, "add_dll_directory"):
    # Windows
    with os.add_dll_directory(OPENSLIDE_PATH):
        import openslide
    print(openslide.__version__)
else:
    print("nope")
    import openslide

1.3.1


In [53]:
import xml.etree.ElementTree as ET
from asap_loader.annotation.objects.rectangle import Rectangle
from asap_loader.annotation.objects.group import Group
from pathlib import Path
from os import listdir

ROOT = "C:\\Users\\MS\\Desktop\\IDEAS\\Projekty\\CD34\\Dane\\CD34_z_adnotacjami"

filenames = listdir(ROOT)
# filename = "22_21450A1_7"
filename = "22_09617A1_7"
xml_path = Path(ROOT, f"{filename}.xml")

tree = ET.parse(xml_path)
root = tree.getroot()

In [67]:
def get_annotation_groups(root):
    # group_names = set()
    groups = []

    for child in root.find("./AnnotationGroups"):
        group_data = {}

        # group_names.add(child.attrib["Name"])
        group_data["name"] = child.attrib["Name"]
        group_data["color"] = child.attrib["Color"]
        group_data["group"] = (
            None
            if child.attrib["PartOfGroup"] == "None"
            else child.attrib["PartOfGroup"]
        )

        group = Group(**group_data)
        groups.append(group)

    return groups

groups = get_annotation_groups(root)
print(len(groups))

5


In [55]:
import pandas as pd

df = pd.read_excel("..\data\średnice.xlsx")
# print(df.dtypes)

df_avg = df[["Typ komórki", "Średnica Najdłuższej", "Średnica Najszerszej"]].groupby(
    "Typ komórki"
).mean()

df_min = (
    df[["Typ komórki", "Średnica Najdłuższej", "Średnica Najszerszej"]]
    .groupby("Typ komórki")
    .min()
)

df_max = (
    df[["Typ komórki", "Średnica Najdłuższej", "Średnica Najszerszej"]]
    .groupby("Typ komórki")
    .max()
)


df_q75 = (
    df[["Typ komórki", "Średnica Najdłuższej", "Średnica Najszerszej"]]
    .groupby("Typ komórki")
    .quantile(0.75)
    .rename(
        columns={
            "Średnica Najdłuższej": "d_max_q75",
            "Średnica Najszerszej": "d_min_q75",
        }
    )
)

# df_out = pd.merge([df_avg, df_min, df_max, df_q75], on="Typ komórki")

df_q75

,d_max_q75,d_min_q75
Typ komórki,,
Komórki,10.7190,NaN
Megakariocyty,21.7125,NaN
Mieloblasty,12.2700,NaN
Naczynia,42.2000,11.455


In [56]:
!uv pip install shapely

Using Python 3.10.18 environment at: C:\Users\MS\Desktop\IDEAS\Projekty\CD34\Repozytoria\slide_tiling-0.1.0\.venv
Audited 1 package in 3ms


In [65]:
from asap_loader.annotation.objects.rectangle import Rectangle
from asap_loader.annotation.objects.polygon import Polygon
from asap_loader.annotation.objects.point import Point

group_dict = {gr.name: gr for gr in groups}
print(group_dict)


for annotation in root.find("./Annotations"):
    if annotation.attrib["Type"] == "Dot":
        coord = annotation.find("./Coordinates/Coordinate")
        point = Point(
            group=group_dict[annotation.attrib["PartOfGroup"]],
            name=annotation.attrib["Name"],
            color=annotation.attrib["Color"],
            x=float(coord.attrib["X"]),
            y=float(coord.attrib["Y"]),
        )
        # print(f"dot: {point}")

    elif annotation.attrib["Type"] == "Polygon":
        coords = annotation.findall("./Coordinates/Coordinate")

        # for coord in coords
        verticies = [(float(coord.attrib['X']), float(coord.attrib['Y'])) for coord in coords]
        polygon = Polygon(
            group=group_dict[annotation.attrib["PartOfGroup"]],
            name=annotation.attrib["Name"],
            color=annotation.attrib["Color"],
            verticies=verticies,
        )
        print(f"polygon: {polygon}")

    elif annotation.attrib["Type"] == "Spline":
        print("spline")

    elif annotation.attrib["Type"] == "PointSet":
        print("points set")

    elif annotation.attrib["Type"] == "Rectangle":
        print("rectangle")

    else:
        assert False

{'Mieloblasty': Group(name='Mieloblasty', color='#00aa00', group=None, members=[Point(name='Annotation 3', color='#F4FA58', group=..., x=50230.9023, y=120242.703), Point(name='Annotation 4', color='#F4FA58', group=..., x=50254.0117, y=120283.906), Point(name='Annotation 5', color='#F4FA58', group=..., x=50212.8203, y=120286.914), Point(name='Annotation 6', color='#F4FA58', group=..., x=50132.4414, y=120293.945), Point(name='Annotation 7', color='#F4FA58', group=..., x=50030.9648, y=120312.031), Point(name='Annotation 8', color='#F4FA58', group=..., x=50173.6367, y=120240.695), Point(name='Annotation 9', color='#F4FA58', group=..., x=50290.1836, y=120279.883), Point(name='Annotation 10', color='#F4FA58', group=..., x=50331.375, y=120157.305), Point(name='Annotation 11', color='#F4FA58', group=..., x=50414.7656, y=120215.578), Point(name='Annotation 12', color='#F4FA58', group=..., x=50440.8906, y=120245.719), Point(name='Annotation 13', color='#F4FA58', group=..., x=50458.9766, y=120261

In [68]:
for poly in group_dict["Naczynia"].members:
    print("ok")

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
